In [1]:
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

D:\Anaconda3\envs\crawler\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\envs\crawler\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\envs\crawler\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\envs\crawler\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: P

In [2]:
#load data
digits = load_digits()
X = digits.data
y = digits.target
print(X.shape)
print(y[:10])
y = LabelBinarizer().fit_transform(y)
print(y[:10])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

(1797, 64)
[0 1 2 3 4 5 6 7 8 9]
[[1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 1]]


In [3]:
def add_layer(inputs, in_size, out_size, n_layer, activation_function=None):
    layer_name = 'layer%s' %n_layer
    with tf.name_scope('layer'):
        with tf.name_scope('Weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]))
            #tf.summary.histogram(layer_name+'/weights',Weights)
        with tf.name_scope('biaes'):
            biaes = tf.Variable(tf.zeros([1, out_size])) + 0.1  #bias一般初始不为0
            #tf.summary.histogram(layer_name+'/biaes',biaes)
        with tf.name_scope('Wx_plus_bias'):
            Wx_plus_bias = tf.matmul(inputs, Weights) + biaes
            Wx_plus_bias = tf.nn.dropout(Wx_plus_bias, keep_prob)   #drop掉50%的结果
            #tf.histogram_summary(layer_name+'/Wx_plus_bias',Weights)
        if activation_function is None:
            outputs = Wx_plus_bias
        else:
            outputs = activation_function(Wx_plus_bias)
        tf.summary.histogram(layer_name+'/outputs',outputs)
        return outputs


# define placeholder for inputs to network
keep_prob = tf.placeholder(tf.float32)  #保持不被drop掉
xs = tf.placeholder(tf.float32, [None, 64]) 
ys = tf.placeholder(tf.float32, [None, 10])

#add output layer
l1 = add_layer(xs, 64, 50, 'l1',activation_function=tf.nn.tanh)
prediction = add_layer(l1, 50, 10, 'l2', activation_function=tf.nn.softmax)


#the loss between prediction and real data
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                              reduction_indices=[1]))       # loss
tf.summary.scalar('loss', cross_entropy)
train_step = tf.train.GradientDescentOptimizer(0.6).minimize(cross_entropy)

sess = tf.Session()
merged = tf.summary.merge_all()
#summary writer goes in here
#writer = tf.train.write_graph(sess.graph,"logs/",name='graph')
train_writer = tf.summary.FileWriter("logs/train",sess.graph)
test_writer = tf.summary.FileWriter("logs/test",sess.graph)

sess.run(tf.initialize_all_variables())


for i in range(500):
    sess.run(train_step, feed_dict={xs: X_train, ys:y_train, keep_prob:1})
    if i % 50 == 0:
        #record loss
        train_result = sess.run(merged, feed_dict={xs: X_train, ys: y_train, keep_prob:1}) #记录时不要drop到任何东西
        test_result = sess.run(merged, feed_dict={xs: X_test, ys:y_test, keep_prob:1})
        train_writer.add_summary(train_result, i)
        train_writer.add_summary(test_result, i)
        

Instructions for updating:
Use `tf.global_variables_initializer` instead.
